In [ ]:
import sqlite3
conn = sqlite3.connect('enterprise.db')
curs = conn.cursor()
curs.execute('''CREATE TABLE zoo (critter VARCHAR(20) PRIMARY KEY, count INT, damages FLOAT)''')


In [ ]:
curs.execute('INSERT INTO zoo VALUES("duck",5,0.0)')
curs.execute('INSERT INTO zoo VALUES("dear",2,1000.0)')

In [ ]:
ins = 'INSERT INTO zoo (critter, count, damages) VALUES(?, ?, ?)'
curs.execute(ins, ('weasel', 1, 2000.0))

In [ ]:
curs.execute('SELECT * FROM zoo')
rows = curs.fetchall()
print(rows)

In [ ]:
curs.execute('SELECT * FROM zoo ORDER BY count')
rows = curs.fetchall()
print(rows)

In [ ]:
curs.execute('SELECT * FROM zoo ORDER BY count DESC')
rows = curs.fetchall()
print(rows)

In [ ]:
curs.execute('''SELECT * FROM zoo WHERE
damages = (SELECT MAX(damages) FROM zoo)''')
rows = curs.fetchall()
print(rows)

In [1]:
import sqlalchemy as sa
conn = sa.create_engine('sqlite://')
conn.execute("""
CREATE TABLE zoo
(critter VARCHAR(20) PRIMARY KEY,
count INT,
damages FLOAT)""")
ins = 'INSERT INTO zoo (critter, count, damages) VALUES (?,?,?)'
conn.execute(ins, 'duck', 10, 0.0)
conn.execute(ins, 'bear', 2, 1000.0)
conn.execute(ins, 'weasel', 1, 2000.0)

rows = conn.execute('SELECT * FROM zoo')
print(rows)

for row in rows:
    print(row)


('duck', 10, 0.0)
('bear', 2, 1000.0)
('weasel', 1, 2000.0)


In [2]:
import sqlalchemy as sa
conn = sa.create_engine('sqlite://')
meta = sa.MetaData()
zoo = sa.Table('zoo', meta,
    sa.Column('critter', sa.String, primary_key=True),
    sa.Column('count', sa.Integer),
    sa.Column('damages', sa.Float)
)
meta.create_all(conn)

conn.execute(zoo.insert(('bear', 2, 1000.0)))
conn.execute(zoo.insert(('weasel', 1, 2000.0)))
conn.execute(zoo.insert(('duck', 10, 0)))

result = conn.execute(zoo.select())
rows = result.fetchall()
print(rows)

[('bear', 2, 1000.0), ('weasel', 1, 2000.0), ('duck', 10, 0.0)]


In [ ]:
import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

conn = sa.create_engine('sqlite:///zoo.db')
Base = declarative_base()
class Zoo(Base):
    __tablename__ = 'zoo'
    critter = sa.Column('critter', sa.String, primary_key=True)
    count = sa.Column('count', sa.Integer)
    damages = sa.Column('damages', sa.Float)
    def __init__(self, critter, count, damages):
        self.critter = critter
        self.count = count
        self.damages = damages
    def __repr__(self):
        return "Zoo({}, {}, {})>".format(self.critter, self.count, self.damages)

Base.metadata.create_all(conn)
first = Zoo('duck', 10, 0.0)
second = Zoo('bear', 2, 1000.0)
third = Zoo('weasel', 1, 2000.0)

Session = sessionmaker(bind=conn)
session = Session()

session.add(first)
session.add_all([second, third])
session.commit()